In [ ]:
# shttps://happy-obok.tistory.com/5

In [1]:
import pandas as pd
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [5]:
import konlpy

In [2]:
# Dataset 다운로드 및 불러오기
# !wget -O .cache/ratings_train.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
# !wget -O .cache/ratings_test.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt

dataset_train = nlp.data.TSVDataset(".cache/ratings_train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset(".cache/ratings_test.txt", field_indices=[1,2], num_discard_samples=1)

In [99]:
# 띄어쓰기
# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [100]:
from pykospacing import Spacing
spacing = Spacing()

In [116]:
idx = 10001
dataset_train[idx][0], spacing(dataset_train[idx][0]) 

('카리스마작렬~역대퍼니셔시리즈1.2.3中가장좋은액션수작!!', '카리스마 작렬~역대 퍼니셔 시리즈 1.2.3中 가장 좋은 액션 수작!!')

In [ ]:
from konlpy.tag import Okt
from tqdm import tqdm

okt = Okt()
nouns_dt = []

stop_words = ['뭐', '더' '좀', '것', '진짜', '이', '거', '점', '완전', '왜', '게', '나', '역시', '수', '내', '저', '말', '이건', '좀', '그냥',
              '영화', '보고', '걸', '개', '안', '볼', '더', '정도', '정말', '그것', '구먼', '구만', '왜케', '때문', '다시']

for dt in tqdm(dataset_train):
    parsed_nouns = okt.nouns(spacing(dt[0]))
    parsed_nouns = [n for n in parsed_nouns if (n not in stop_words) and (len(n) > 1)]
    nouns_dt.append(parsed_nouns)

 99%|█████████▉| 148328/150000 [1:15:54<00:48, 34.19it/s]

In [120]:
nouns_dt[:10]

[['더빙', '목소리'],
 ['포스터', '초딩', '오버', '연기'],
 ['재밓었다', '추천'],
 ['교도소', '이야기', '재미', '평점', '조정'],
 ['페그', '연기', '스파이더맨', '커스틴', '던스트'],
 ['걸음', '초등학교', '학년', '생인', '별반', '개도'],
 ['원작', '긴장감', '제대로'],
 ['반개',
  '응경',
  '길용우',
  '생활',
  '해도',
  '납치',
  '감금',
  '반복',
  '반복',
  '드라마',
  '가족',
  '연기',
  '사람'],
 ['액션', '재미'],
 ['평점', '헐리우드']]

In [121]:
import gensim
id2word = gensim.corpora.Dictionary(nouns_dt)
id2word.filter_extremes(no_below = 100) #20회 이하로 등장한 단어는 삭제

texts = nouns_dt
corpus = [id2word.doc2bow(text) for text in texts]

In [122]:
lda = gensim.models.LdaModel(
    corpus,
    id2word=id2word,
    num_topics=3)

In [123]:
lda.show_topics()

[(0,
  '0.050*"최고" + 0.035*"사람" + 0.034*"내용" + 0.030*"감독" + 0.027*"시간" + 0.024*"하나" + 0.023*"작품" + 0.018*"최악" + 0.016*"명작" + 0.015*"주인공"'),
 (1,
  '0.063*"연기" + 0.045*"감동" + 0.038*"재미" + 0.038*"배우" + 0.031*"쓰레기" + 0.021*"별로" + 0.015*"보기" + 0.013*"음악" + 0.012*"수준" + 0.012*"가슴"'),
 (2,
  '0.042*"평점" + 0.036*"생각" + 0.035*"드라마" + 0.029*"스토리" + 0.018*"마지막" + 0.017*"처음" + 0.017*"장면" + 0.015*"이야기" + 0.014*"지금" + 0.011*"여자"')]

In [124]:
lda.inference(corpus)

(array([[2.3332226 , 0.33334556, 0.33333832],
        [0.33334157, 4.333152  , 0.333342  ],
        [0.333339  , 1.3333063 , 0.33333522],
        ...,
        [0.3334063 , 0.3333631 , 1.3330377 ],
        [0.3333497 , 0.33336353, 1.3331486 ],
        [1.3111647 , 1.3514769 , 1.3369106 ]], dtype=float32),
 None)